In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math

from Clust.clust.ML.regression_JS import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.ML.common.common import p3_training as p3
from Clust.clust.transformation.purpose.machineLearning import LSTMData

# Set Param

In [2]:
param = {
    "d_feat" : 6, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

shuffle = False
model_file_path = "./ada_rnn_model/test_adarnn_model_by_origin_data"

# get Data

In [3]:
# read original data
data_pickle = pd.read_pickle("original_data.pkl")
original_data = data_pickle["Dongsi"]

In [4]:
len(original_data), original_data[0].shape, original_data[2].shape

(3, (1460, 24, 6), (1460,))

In [5]:
# original data 에서 data_x, data_y 나누기
data_x = original_data[0]
data_y = original_data[2]

In [6]:
# set time
original_data_start_time = datetime.datetime(2013, 3, 1, 0, 0)

## test data time
test_start_time = '2016-11-2 00:00:00'
test_end_time = '2017-2-28 23:00:00'

In [7]:
# split train & validation data -> get train_x, train_y, valid_x, valid_y
index_start=(pd.to_datetime(test_start_time) - original_data_start_time).days
index_end=(pd.to_datetime(test_end_time) - original_data_start_time).days

test_x=data_x[index_start: index_end + 1]
test_y=data_y[index_start: index_end + 1]

In [8]:
test_x.shape, test_y.shape

((118, 24, 6), (118,))

# Data Loader

In [9]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'d_feat': 6, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [10]:
test_loader = adarnn.create_testloader(test_x, test_y, shuffle)

In [11]:
list(test_loader)[0][0].shape

torch.Size([64, 24, 6])

In [12]:
list(test_loader)[0][1].shape

torch.Size([64])

In [13]:
len(test_loader)

2

# Testing

In [14]:
## model load
adarnn.load_model(model_file_path)

In [15]:
## test loader 제거 작업 시작
test_loss, test_loss_l1, test_loss_r = adarnn.test(test_loader)

Test: 100%|███████████████████████████████████████| 2/2 [00:00<00:00, 10.51it/s]

[2023-03-16 09:44:25] - Finished.


In [16]:
test_loss

0.003794912830926478

In [17]:
test_loss_l1

0.04521500878036022

In [18]:
test_loss_r

tensor(0.0222, device='cuda:0')